# pyNBL: Basketball Statistic System for Australian NBL

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ssardina/pynbl/HEAD)

This notebook **incrementally** builds a set of stat tables from NBL Basketball Games:

1. A table of games played, with team names, points, venue, etc.
2. A stat table of _stint lineups_ (advance) statistics for each game and each team. A **stint** is a lineup of players who play together in different interval periods across the game. This table will contain the stints for each team from the play-by-play data and compute various statistics for those stints.

Tables will be saved in CSV and Excel formats as well as in [Pickle format](https://docs.python.org/3/library/pickle.html) for later recovery as Panda DataFrames.


The data comes as a raw JSON file using the game id (e.g., `2087737`):

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [8]:
# Let's first load all required packages...
import os
import pandas as pd
import numpy as np
import dtale

from config import *
import bball_stats
import tools

# Overwrite default DATA_DIR folder where tables will be dumped
DATA_DIR='data-21_22/'

# Files containing the existing tables stored in disk already
file_stats_df = os.path.join(DATA_DIR, "stats_df.pkl")
file_games_df = os.path.join(DATA_DIR, "games_df.pkl")

## 1. Define games to scrape and saved data

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

In [12]:
# Games to be computed

#   Format: (game id, round number)
games_21_22 = [(1976446, 1), (1976447, 1), (1976448, 1), (1976452, 1), (1976454, 1), (2004608, 1), (2004609, 2), (1976449, 2), (1976451, 2), (1976453, 2), (1976455, 2), (1976458, 2), (1976456, 2), (1976457, 3), (1976459, 3), (1976460, 3), (1976461, 3), (1976462, 3), (1976463, 3), (1976464, 3), (2004610, 3), (1976465, 3), (1976468, 4), (1976469, 4), (1976474, 5), (1976473, 5), (1976482, 6), (2036215, 7), (2031329, 7), (2031330, 7), (2031332, 7), (2031333, 7), (2031334, 7), (2031335, 8), (2031336, 8), (2031337, 8), (2031338, 8), (2031340, 8), (2031341, 8), (2046695, 8), (2046696, 8), (2046697, 9), (2031342, 9), (2031343, 9), (2031344, 9), (2031345, 9), (2031346, 9), (2031347, 9), (2046698, 10), (2046700, 10), (2046701, 10), (2046702, 10), (2046703, 10), (2046704, 10), (2046706, 10), (2046707, 11), (2046709, 11), (2046710, 11), (2046711, 11), (2046712, 11), (2046713, 11), (2051763, 11), (2053811, 12), (2053812, 12), (2053813, 12), (2053814, 12), (2053815, 12), (2053816, 12), (2053817, 12), (2053818, 13), (2053819, 13), (2053820, 13), (2053821, 13), (2053822, 13), (2053823, 13), (2053824, 13), (2053825, 13), (2056454, 14), (2056455, 14), (2056457, 14), (2056458, 14), (2056461, 14), (2056462, 14), (2056460, 14), (2056463, 15), (2056464, 15), (2056466, 15), (2056467, 15), (2056469, 15), (2056471, 15), (2056472, 15), (2056473, 15), (2065653, 16), (2065654, 16), (2065655, 16), (2065656, 16), (2065657, 17), (2065658, 17), (2065659, 17), (2069165, 16), (2069166, 16), (2069167, 16), (2069168, 17), (2069169, 17), (2069170, 17), (2069171, 17), (2069172, 17), (2069175, 18), (2069177, 18), (2069179, 18), (2069181, 18), (2069183, 18), (2069184, 18), (2069186, 18), (2069187, 18), (2069191, 18), (2069192, 19), (2069194, 19), (2069196, 19), (2069199, 19), (2069202, 19), (2069203, 19), (2069204, 19), (2069190, 19), (2069193, 19), (2069195, 20), (2069197, 20), (2069198, 20), (2069200, 20), (2069201, 20), (2069205, 20), (2069173, 20), (2069174, 21), (2069176, 21), (2069178, 21), (2069180, 21), (2069182, 21), (2069188, 21), (2069189, 20)]

games_22_23 = [(2141127, 0), (2135117, 0), (2134935, 0), (2141126, 0), (2135116, 0), (2122060, 0), (2122059, 0)]

# games = [1976463]   # game with no "bugs" in subs
games = games_21_22


# Set to True to re-compute from scratch all tables
reload = False

SyntaxError: unmatched ']' (2970848630.py, line 4)

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, computes stats and game info, and adds them to the initial tables of stats and games.

We start by loading all saved previous games, if any, as we want to append to that database (and we don't want to recompute them).

In [7]:
# Load tables from saved files (if any)
init_stats_df = None
game_df = None
if os.path.exists(file_stats_df) and not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading initial stats df: {file_stats_df}")
    init_stats_df = pd.read_pickle(file_stats_df)
    game_df = pd.read_pickle(file_games_df)
    existing_games = init_stats_df.game_id.unique()
else:
    existing_games = []

init_stats_df.sample(3)
# init_stats_df['lineup'].apply(lambda x: len(x) > 5)
# init_stats_df.loc[5,'lineup']
# init_stats_df.loc[5]

Loading initial stats df: data-22_23/stats_df.pkl


,game_id,tno,team,game1,round1,game2,round2,stint,poss,ortg,...,tov_bh_opp,tov_bp_opp,tov_ofoul_opp,tov_3sec_opp,tov_8sec_opp,tov_24sec_opp,opp_fga_blocked_opp,lineup,intervals,mins
19,2141127,1,Perth Wildcats,0,0,0,0,20,4.0,0.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[B. Cotton, B. Manek, L. Travers, M. Norton, T...","[(4, 00:08:05, 00:06:16)]",1.816667
165,2135116,2,Brisbane Bullets,0,0,0,0,2,5.0,180.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[D. Mitchell, D. Walker, H. Froling, J. Cadee,...","[(1, 00:07:49, 00:05:50), (3, 00:06:23, 00:06:...",2.300000
105,2134935,2,Illawarra Hawks,0,0,0,0,2,6.0,116.67,...,0.0,0.0,1.0,0.0,0.0,0.0,20.0,"[A. Mudronja, D. Deng, J. Robinson, S. Froling...","[(1, 00:05:59, 00:05:08), (2, 00:07:31, 00:05:...",3.800000


It is now time to process each game:

In [ ]:
stats_dfs = [init_stats_df] if init_stats_df is not None else []
games_data = []
for game in games:
    # check if a game item includes game and round numbers for the two teams; otherwise assign np.nan
    if isinstance(game, tuple):
        game_id, game1, round1, game2, round2 = game
    else:
        game_id = game
        game1 = round1 = game2 = round2  = np.nan

    # don't scrape game data if already loaded from file
    if game_id in existing_games:
        print(f"Game {game_id} is already in table")
        continue
    print(f"Computing game {game_id}...")

    # MAIN STEP: scrape and compute the actual stats for the game
    result = bball_stats.build_game_stints_stats_df(game_id)
    df = result['stint_stats_df']   #  this is basically what we care, the stint stats
    team1, team2 = result['teams']

    # extract date of game from HTML page
    try:
        game_info = tools.get_game_info(game_id)
    except:
        game_info = { "venue" : np.nan, "date": np.nan}
    print(f"\t .... done: {team1[0]} ({team1[1]}) vs {team2[0]} ({team2[1]}) on {game_info['date']}")

    # fill game info
    df.insert(0, 'game_id', game_id)
    df.insert(3, 'game1', game1)
    df.insert(4, 'round1', round1)
    df.insert(5, 'game2', game2)
    df.insert(6, 'round2', round2)
    stats_dfs.append(df)

    # build game dataframe table
    games_data.append({"game_id": game_id,
                        "date" : game_info['date'],
                        "team1": team1[0], "team2": team2[0],
                        "s1": team1[1], "s2": team2[1],
                        "game1": game1, "round1": round1,
                        "game2": game2, "round2": round2,
                        "winner": 1 if team1[1] > team2[1] else 2,
                        "venue" : game_info["venue"]}
                      )

# put all dfs for each game together into a single dataframe
stats_df = pd.concat(stats_dfs)
stats_df.reset_index(inplace=True, drop=True)
stats_df.sample(5)

if game_df is not None:
    games_df = pd.concat([game_df, pd.DataFrame(games_data)])
    games_df.reset_index(inplace=True, drop=True)
else:
    games_df = pd.DataFrame(games_data)

print("All games extracted!")

If we want we can do some sanity checks, before saving to disk:

In [ ]:
dtale.show(games_df)

In [ ]:
import random
print("The shape of stats_df is:", stats_df.shape)
stats_cols = list(stats_df.columns[8:53])
print("Stats cols:", stats_cols)

# build columns we want to show
cols = ['game_id' , 'tno', 'team', 'stint']
rnd_cols = random.sample(stats_cols, 8)
rnd_cols.extend([f"{x}_opp" for x in rnd_cols])
cols.extend(rnd_cols)

# show some sample of stats computed
stats_df[cols].sample(5)

Sanity check that `(ortg, drtg)` (offensive/defensive rate goal) should mirror `(drtg_opp, ortg_opp)` (opponent offensive/defensive rate goal)):

In [ ]:
# (ortg, drtg) should mirror (drtg_opp, ortg_opp)
stats_df.iloc[4][['game_id' , 'team', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [ ]:
import datetime
import os

stats_df.to_pickle(os.path.join(DATA_DIR, "stats_df.pkl"))
games_df.to_pickle(os.path.join(DATA_DIR, "games_df.pkl"))

stats_df.to_csv(os.path.join(DATA_DIR, "stats_df.csv"), index=False)
games_df.to_csv(os.path.join(DATA_DIR, "games_df.csv"), index=False)

with pd.ExcelWriter(os.path.join(DATA_DIR, 'stats_df.xlsx')) as writer:
    stats_df.to_excel(writer, sheet_name='STATS', index=False)
    games_df.to_excel(writer, sheet_name='GAMES', index=False)
games_df.to_excel(os.path.join(DATA_DIR, "games_df.xlsx"))


now = datetime.datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
print(f"Finished saving at time: {date_time}")

## 4. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [ ]:
dtale.show(stats_df)
# dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

## 5. Some checks...

Check if a stint lineup has more than 5 players! It could happen:

1. Game 2031329, player H. Besson comes out (wrongly?) at 3rd period min 10:00 but he keeps playing and then goes out again at 7:33.

In [ ]:
stats_df.shape
mask = stats_df['lineup'].apply(lambda x: len(x) != 5)
stats_df[mask]

# stats_df.iloc[941][['game_id', 'lineup']]